<a href="https://colab.research.google.com/github/dramamine/ml-style-transfer-experimentation/blob/multi-preview/preview_mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preview Image Styles

TODO:  Link to inspiration demos and credit sources


In [ ]:
# Here's the rough plan:

# V0 - take two images, run them at 1x1 ✅
# V1 - take two directories of 2 images, run them at 1x1, display in a grid with originals

# Stretch goals
# - add "random sample" mode for pulling handful of imgs from dir
# - directory chooser / file browser
# - add "pick your images" mode

In [ ]:
#@title 0. Get utils code from github 🐙🐱

# Here's how to import from our github for these files!
branch_name = 'main' #@param

# Start from /content/ dir, clone git repo
%cd /content
%rm -rf ml-style-transfer-experimentation
!git clone https://github.com/dramamine/ml-style-transfer-experimentation

# cd into git repo, change branch
%cd ml-style-transfer-experimentation
!git checkout $branch_name

# import python files
from lib.utils import test_a_python_import_from_github
from lib.utils import StepTimer
test_a_python_import_from_github()

#
# Import Style Transfer Utils
#
import lib.style_transfer_utils as sxu
print('')
print('👍 imported style transfer utils 👍') if sxu else print('💀 ERROR 💀')
print('')

# back to base directory
%cd /content


In [ ]:
#@title 1. Install Dependencies 🧰
#@markdown Install dependencies. ***Probably don't modify this?*** The setup should not take more than two minutes. 
print("Installing tf-nightly...")
!pip uninstall -q -y tensorflow 
!pip install -q tf-nightly

from IPython.display import clear_output
import matplotlib.pyplot as plt
import tensorflow as tf
 
clear_output()

print("tf-nightly installed.")

# Download the style bottleneck and transfer networks
print('Downloading the model files...')

style_predict_path = tf.keras.utils.get_file('style_predict.tflite', 'https://tfhub.dev/sayakpaul/lite-model/arbitrary-image-stylization-inceptionv3/int8/predict/1?lite-format=tflite')
style_transform_path = style_transform_path = tf.keras.utils.get_file('style_transform.tflite', 'https://tfhub.dev/sayakpaul/lite-model/arbitrary-image-stylization-inceptionv3/int8/transfer/1?lite-format=tflite')

print('Model files downloaded...')

# other deps
from IPython.display import display
import ipywidgets as widgets

import numpy as np
import io
import os

print('You are all set!')

In [ ]:
#@title 2. Mount Google Drive

print("Mounting google drive...")
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from PIL import Image
import os
def load_or_render_preview_cell(content_image_path, style_image_path, blend_ratio, base_dir, output_dir):
  params = dict(
    # inputs
    content_image_path=content_image_path,
    style_image_path=style_image_path,
    content_blending_ratio=blend_ratio,
    # preview param settings
    rows=1,
    cols=1,
    use_tiled_style_image=False,
    use_fluid_blend=False,
    edge_size=8,
    magnitude=2,
    squeeze=0,
    # file locations
    drive_base=base_dir,
    output_directory=output_dir,
  )

  # Check to see if we already rendered this preview, and return it if so
  target_filename = output_dir + sxu.get_output_filename(**params)
  if os.path.isfile(target_filename):
    return Image.open(target_filename)

  # Render the preview and log the timing
  with StepTimer('[[TIMING]] Render preview image'):
    return sxu.run(**params)

In [ ]:
from PIL import Image
# square and center the image by removing extra height or width
def square_crop_and_center(img):
  width, height = img.size
  new_width = new_height = min(width, height)

  left = (width - new_width)/2
  top = (height - new_height)/2
  right = (width + new_width)/2
  bottom = (height + new_height)/2

  return img.crop((left, top, right, bottom))

In [ ]:
#@title 3. Run It
import matplotlib.pyplot as plt
from PIL import Image

#
# Preview Grid Parameters
#
gdrive_basedir = '/content/drive/MyDrive/images/' #@param

content_folder = 'content' #@param
style_folder = 'style' #@param

content_blending_ratio = 0.2 #@param {type:"slider", min:0, max:1, step:0.1}

preview_width = 15 #@param {type:"slider", min:5, max:15, step:1}
preview_height = 15 #@param {type:"slider", min:5, max:15, step:1}

gdrive_outdir = '/content/drive/MyDrive/images/previews/' #@param

#
# Render Preview Grid
#

content_imgs = os.listdir(gdrive_basedir + content_folder)
style_imgs =  os.listdir(gdrive_basedir + style_folder)

content_len = len(content_imgs)
style_len = len(style_imgs)

# initialize preview grid
preview_grid, subplots = plt.subplots(content_len+1, style_len+1)

# remove ALL grid axes
for ii in range(content_len+1):
  for jj in range(style_len+1):
    subplots[ii, jj].axis('off')

# show content and style images (crop to content size)
for ii in range(content_len):
  for jj in range(style_len):
    subplots[ii+1, 0].imshow(
      square_crop_and_center(
        Image.open(
          gdrive_basedir + 
          content_folder + '/' +
          content_imgs[ii]
        )
      )
    )
    subplots[0, jj+1].imshow(
      square_crop_and_center(
        Image.open(
          gdrive_basedir + 
          style_folder + '/' +
          style_imgs[jj]
        )
      )
    )

# render preview grid
for ii in range(content_len):
  for jj in range(style_len):
    content_image_path = f'{content_folder}/{content_imgs[ii]}'
    style_image_path = f'{style_folder}/{style_imgs[jj]}'

    result_plot = subplots[ii+1, jj+1]
    result = load_or_render_preview_cell(
      content_image_path,
      style_image_path,
      content_blending_ratio,
      gdrive_basedir,
      gdrive_outdir
    )
    result_plot.imshow(result)

preview_grid.subplots_adjust(hspace=0.20, wspace=0.05)
preview_grid.set_size_inches(preview_width, preview_height)

preview_grid.savefig(gdrive_outdir + 'preview.jpg')

print('\nPreview Grid')